# 例子 - 使用 `huggingface/transfomers.GPT2LMHeadModel` 进行微调

> ❗ **要点**:
>
> NVIDIA/Megatron-LM 的 `tokenizer` 会在 `128` 对齐的基础上，强制加上 `8` 个特殊 `token`，所以注意 `token` 的偏移！


该 Notebook 的代码大部分来自: <https://github.com/huggingface/transformers/blob/master/examples/run_generation.py>

## 切换工作目录

注意不要搞错!按照实际情况切换！

In [1]:
%cd ..

/home/liuxy/workspace/Megatron-LM


## 环境变量

- 不需要 `CUDA` 设备:

In [2]:
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [3]:
%run finetune_gpt2.py \
    --model_name_or_path checkpoints/hfgpt2-117m-emotion/ \
    --output_dir tmp \
    --tokenizer_model_path data/spm/gpt2_huamei_corpus_bpe_32k_v2.model \
    --do_train \
    --train_data_file data/baike10k.json \
    --fp16

/home/liuxy/miniconda3/envs/Megatron_LM-ipy/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/liuxy/miniconda3/envs/Megatron_LM-ipy/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/liuxy/miniconda3/envs/Megatron_LM-ipy/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)]

RuntimeError: CUDA out of memory. Tried to allocate 96.00 MiB (GPU 0; 10.73 GiB total capacity; 0 bytes already allocated; 84.50 MiB free; 0 bytes cached)